In [2]:
import sys 
import os
if os.getlogin()=='womo1998':
    base='/vegas/scratch/womo1998/'
    sys.path.append("/vegas/users/staff/womo1998/git/pyOMA/")  
else:
    base='/usr/scratch4/sima9999/work/'
    sys.path.append('/usr/wrk/people9/sima9999/git/pyOMA')
import numpy as np
from uncertainty.data_manager import DataManager, HiddenPrints
import logging
logger = logging.getLogger('uncertainty.data_manager')
logger.setLevel(level=logging.DEBUG)
from core.StabilDiagram import StabilCalc
data_manager = DataManager.from_existing(dbfile_in='uq_acqui.nc', 
                                         result_dir=base + 'modal_uq/uq_acqui/')

DEBUG:uncertainty.data_manager:opening existing file uq_acqui.nc


In [3]:
def func(ds):
    '''
    plot all numerical natural frequencies to get a feeling for their distribution
    
    modal parameters:
    for each jid:
        assign each identified mode to the respective numerical
        mark unidentified and wrongly identified
        compute normalized frequency and damping differences (numerical, identified), MAC values
    
    per-channel output variables:
        reduce to mean and coefficient of variation
    '''
#     for name in ds.data_vars:
#         if name.startswith('_'): continue
#         print(name, ds[name].dims)
    
    num_samples = ds.ids.size
    num_modes = ds.modes.size
    num_channels = ds.channels.size
    
    ds['freq_diff'] = (['ids','modes'], np.full(shape=(num_samples, num_modes), fill_value=np.nan))
    ds['damp_diff'] = (['ids','modes'], np.full(shape=(num_samples, num_modes), fill_value=np.nan))
    ds['modal_assurance'] = (['ids','modes'], np.full(shape=(num_samples, num_modes), fill_value=np.nan))
    ds['unp_num'] = (['ids'], np.zeros((num_samples,)))
    ds['unp_id'] = (['ids'], np.zeros((num_samples,)))    
#     skip=True    
    print_ids = ds.ids[np.linspace(0, num_samples-1, 100, dtype=int)]
    #print(print_ids)
    for jid in ds.ids:
        if jid.item() in print_ids: print('.', end='', flush=True)
#         print(jid.item()
#         if jid!='123a8b385cb9':
#             if skip:continue
#         skip=False
        freq_a = ds['frequencies_a'].loc[jid].dropna('modes', 'all')
        freq_id = ds['modal_frequencies'].loc[jid].dropna('modes', 'all')
        damp_a = ds['damping_a'].loc[jid].dropna('modes', 'all')
        damp_id = ds['modal_damping'].loc[jid].dropna('modes', 'all')
        shapes_a = ds['mode_shapes_a'].loc[jid].dropna('modes', 'all').dropna('channels', 'all')
        shapes_id = ds['mode_shapes'].loc[jid].dropna('modes', 'all').dropna('channels', 'all')
        
        ind_num, ind_id, unp_num, unp_id= pair_modes(freq_a, freq_id,
                                                     shapes_a, shapes_id,)
        n_pairs = len(ind_num)
        ds['unp_num'].loc[jid] = unp_num
        ds['unp_id'].loc[jid] = unp_id
        
        freq_diff = (freq_a[ind_num] - freq_id[ind_id])/(0.5 * (freq_a[ind_num] + freq_id[ind_id]))
        ds['freq_diff'].loc[jid][ind_num] = freq_diff
        damp_diff = (damp_a[ind_num] - damp_id[ind_id])
        ds['damp_diff'].loc[jid][ind_num] = damp_diff
        mac_values = np.diag(StabilCalc.calculateMAC(shapes_a[:,ind_num], shapes_id[:,ind_id]))
        ds['modal_assurance'].loc[jid][ind_num] = mac_values       
    
    print('.', end='\n', flush=True)
    
    #ds['this_delta_f'] = ds['freq_diff'].mean(dim='modes')
    #ds['this_delta_d'] = ds['damp_diff'].mean(dim='modes')
    #ds['this_mac'] = ds['modal_assurance'].mean(dim='modes')
    
    ds['duration'] = ds.num_cycles/ds.frequencies_a.min(dim='modes')
    
    #dt_fact=0.01
    #ds['deltat'] = dt_fact / ds.frequencies_a.max(dim='modes')
    #ds['timesteps'] = ds.duration / ds.deltat # N

    ds['all_n_cycl'] = ds.duration * ds.frequencies_a
    ds = ds.rename_dims({'ids':'ids_'})
    ds = ds.stack(ids=('ids_','modes'))
    ds = ds.dropna('ids','all',subset=('freq_diff',))
    ds = ds.reset_index('ids')
    ds = ds.reset_coords(['ids_','modes'])
    
    ds['this_snr_alias'] = ds['snr_alias'].mean(dim='channels')
    ds['this_snr_quant'] = ds['snr_quant'].mean(dim='channels')
    ds['this_snr_db_out'] = ds['snr_db_out'].mean(dim='channels')
    
    return ds
        
def pair_modes(freq_num, freq_ident, 
               shapes_num, shapes_ident, 
               freq_thresh=0.2, mac_thresh=0.8):
    '''
    A function to pair two sets of modes (here: numerical and identified) 
    based on frequency differences and mode shape similarity
    the number of modes in both sets may be different and
    relative complements of both array may be non-empty
    
    a threshold where pairing stops is based on 
        normalized frequency differences AND modal assurance criteria
    
    returns:
        * integer array indexers for both arrays 
          sorted to return ascending frequencies (numerical)
          length represents the number of common modes
        * for both sets a count for the number of unpaired modes
    
    '''
#   print(freq_num, freq_ident)
#     for array in [freq_num, freq_ident, shapes_num, shapes_ident]:
#         print(array)
#         assert not np.any(np.isnan(array))
    shape=(len(freq_num), len(freq_ident))
    delta_matrix = np.ma.array(np.zeros(shape), mask=np.zeros(shape))
    for index, frequency in enumerate(freq_num):
        delta_matrix[index, :] = np.abs(
            (freq_ident - frequency) / (0.5*(freq_ident + frequency)) )
    
    mac_matrix = StabilCalc.calculateMAC(shapes_num, shapes_ident) 
#   might have to be transposed
#   indices and sizes of delta_matrix and mac_matrix should be equal
#   print(delta_matrix.shape, mac_matrix.shape)
#   print(delta_matrix, mac_matrix)
    
    indices_num = []
    indices_ident = []
    delta_values = []
    mac_values = []
    
    while ~np.all(delta_matrix.mask):
        # find index of smallest frequency difference
        row, col = np.unravel_index(
            np.argmin(delta_matrix), delta_matrix.shape)
        
        # if another column contains a minimal value in the same row
        # do not mask the column
        for col_ind in range(delta_matrix.shape[1]):
            if col_ind == col:
                continue
            if delta_matrix[:, col_ind].mask.all():
                continue
            if np.argmin(delta_matrix[:, col_ind]) == row:
                del_col = False
                break
        else:
            del_col = True
            col_ind = col
        # if another row contains a minimal value in the same column
        # do not mask the row
        for row_ind in range(delta_matrix.shape[0]):
            if row_ind == row:
                continue
            if delta_matrix[row_ind, :].mask.all():
                continue
            if np.argmin(delta_matrix[row_ind, :]) == col:
                del_row = False
                break
        else:
            del_row = True
            row_ind = row
            
#         print(row,col,delta_matrix[row,col]<freq_thresh, mac_matrix[row,col]>mac_thresh, del_row,del_col)
        
        if delta_matrix[row,col]<freq_thresh and mac_matrix[row,col]>mac_thresh:#del_col and del_row:
            delta_values.append(delta_matrix[row, col])
            mac_values.append(mac_matrix[row, col])
            delta_matrix[row, :] = np.ma.masked
            delta_matrix[:, col] = np.ma.masked            
            indices_num.append(row)
            indices_ident.append(col)
        
        # we still have do something,
        # else we would stay in an infinite loop here
        elif del_row and del_col:
            delta_matrix[row, :] = np.ma.masked
            delta_matrix[:, col] = np.ma.masked   
        
        # in the case, where we might discard a candidate for a good match for another mode
        # we use the modal assurance criterion to decide which mode to discard
        # counter-intuitively that should be the best matching mode of several candidates
        elif not (del_row and del_col): # one of both must be false
            # both members of the selected pair also have another close match
            # which of the three candidates has the best MAC value?
            best = np.argmax([mac_matrix[row_ind, col], mac_matrix[row, col_ind], mac_matrix[row, col]])
            if best==0:
                # another row (row_ind) contains a minimal value in the same column 
                row=row_ind
            elif best==1:
                # another column (col_ind) contains a minimal value in the same row
                col=col_ind
            else: 
                # initial mode is better candidate
                pass
            delta_matrix[row, :] = np.ma.masked
            delta_matrix[:, col] = np.ma.masked        
        else:
            print(delta_matrix)
            print(col_ind, row_ind, col, row)
            print(delta_matrix[row_ind, col_ind], 
                  delta_matrix[:, col_ind],
                  delta_matrix[row_ind, :])
            raise RuntimeError('Caught in a loop')

    
    # now sort according to ascending numerical frequencies
    sort_inds = np.argsort(freq_num.data[indices_num])

    indices_num = np.array(indices_num)[sort_inds]
    indices_ident = np.array(indices_ident)[sort_inds]
    
    unp_num = [i for i in range(len(freq_num)) if i not in indices_num]
    unp_ident = [i for i in range(len(freq_ident)) if i not in indices_ident]
    
#     print((freq_num[indices_num]-freq_ident[indices_ident])/(0.5*(freq_num[indices_num]+freq_ident[indices_ident])))
#     print(np.array(delta_values)[sort_inds])
#     print('\n')
#     print(mac_matrix[indices_num, indices_ident])
#     print(np.array(mac_values)[sort_inds])
#     print('\n')
#     print(freq_num[indices_num], freq_num[unp_num])
#     print(freq_ident[indices_ident], freq_ident[unp_ident])

    return indices_num, indices_ident, len(unp_num), len(unp_ident)

In [3]:
data_manager.post_process_samples(db='merged',func=func, rw=True)

DEBUG:uncertainty.data_manager:opening existing file uq_acqui.nc
DEBUG:uncertainty.data_manager:opening existing file uq_acqui_out.nc
INFO:uncertainty.data_manager:Applying user-supplied function func to dataset and save results (True)...
.....................................................................................................
DEBUG:uncertainty.data_manager:Saving database to /usr/scratch4/sima9999/work/modal_uq/uq_acqui/_uq_acqui_out.nc
INFO:uncertainty.data_manager:We saved the database outside the safe loop, ensure to rename the database for further use.


/usr/wrk/people9/sima9999/my-python/lib/python3.8/site-packages/xarray/backends/h5netcdf_.py:315: FutureWarning: complex dtypes are supported by h5py, but not part of the NetCDF API. You are writing an HDF5 file that is not a valid NetCDF file! In the future, this will be an error, unless you set invalid_netcdf=True.
  nc4_var = self.ds.create_variable(


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
with data_manager.get_database(database='out', rw=True) as out_ds, data_manager.get_database(database='in', rw=False) as in_ds:
    out_ds['num_cycles']=in_ds['num_cycles']
ds


In [ ]:
with data_manager.get_database(database='out', rw=False) as ds:
    ds


ds['duration'] = ds.num_cycles/ds.frequencies_a.min(dim='modes')

#dt_fact=0.01
#ds['deltat'] = dt_fact / ds.frequencies_a.max(dim='modes')
#ds['timesteps'] = ds.duration / ds.deltat # N

ds['all_n_cycl'] = ds.duration * ds.frequencies_a
ds = ds.rename_dims({'ids':'ids_'})
#display(ds)
ds = ds.stack(ids=('ids_','modes'))
#display(ds.ids_)
ds = ds.dropna('ids','all')
#display(ds)
ds = ds.reset_index('ids')
display(ds)

In [49]:
with data_manager.get_database(database='processed', rw=True) as ds:
    #ds.unp_num.attrs = {'dist_type': 'integers', 'dist_params': np.array([1,9])}
    #ds.unp_id.attrs = {'dist_type': 'integers', 'dist_params': np.array([0,92])}
    #ds.dec_rate.attrs = {'dist_type': 'integers', 'dist_params': np.array([6,83])}
    #ds['window'] = ('ids', ds.window.data=='hann', {'dist_type': 'integers', 'dist_params': np.array([0,2])})
    #ds['numtaps_fact'] = ds.numtaps / ds.dec_rate #= ds.reset_coords(['ids_','modes'])
    #ds['nyq_rat'] = ds.fs/ds.cutoff
    #ds.to_netcdf('/usr/scratch4/sima9999/work/modal_uq/uq_acqui/uq_acqui_processed2.nc',engine='h5netcdf')
    pass
display(ds)

DEBUG:uncertainty.data_manager:opening existing file uq_acqui_processed.nc
DEBUG:uncertainty.data_manager:Acquired lock on /usr/scratch4/sima9999/work/modal_uq/uq_acqui/uq_acqui_processed.nc.lock
DEBUG:uncertainty.data_manager:Saving database to /usr/scratch4/sima9999/work/modal_uq/uq_acqui/uq_acqui_processed.nc


<xarray.Dataset>
Dimensions:              (ids: 6175, channels: 10)
Coordinates:
    ids_                 (ids) int64 0 0 0 0 0 0 0 ... 998 999 999 999 999 999
    modes                (ids) int64 1 2 3 4 6 7 8 1 2 3 ... 4 5 6 7 8 0 1 2 4 5
Dimensions without coordinates: ids, channels
Data variables: (12/42)
    _exceptions          (ids) object '' '' '' '' '' '' '' ... '' '' '' '' '' ''
    _runtimes            (ids) float64 77.62 77.62 77.62 ... 34.58 34.58 34.58
    all_n_cycl           (ids) float64 4.215e+03 7.024e+03 ... 8.159e+03
    bits                 (ids) float64 8.0 8.0 8.0 8.0 ... 32.0 32.0 32.0 32.0
    cutoff               (ids) float64 220.3 220.3 220.3 ... 210.7 210.7 210.7
    damp_diff            (ids) float64 -5.094 -3.763 -3.649 ... -4.043 -3.395
    ...                   ...
    this_snr_alias       (ids) float64 4.474e+05 4.474e+05 ... 3.865e+05
    this_snr_db_out      (ids) float64 -21.34 -21.34 -21.34 ... -22.01 -22.01
    this_snr_quant       (ids) float64 10.89 10.89 10.89 ... 156.6 156.6 156.6
    unp_id               (ids) float64 58.0 58.0 58.0 58.0 ... 34.0 34.0 34.0
    unp_num              (ids) float64 2.0 2.0 2.0 2.0 2.0 ... 4.0 4.0 4.0 4.0
    window               (ids) int8 1 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    date:         datetime.date(2021, 9, 7)
    dbfile_in:    uq_acqui.nc
    dbfile_out:   uq_acqui_out.nc
    entropy:      300507817
    result_dir:   /usr/scratch4/sima9999/work/modal_uq/uq_acqui/
    title:        uq_acqui
    working_dir:  /dev/shm/womo1998/

In [4]:
with data_manager.get_database(database='out', rw=False) as ds:
    ds

DEBUG:uncertainty.data_manager:opening existing file uq_acqui_out.nc


In [10]:
ds.where(ds.snr_db_out>0, drop=True)

<xarray.Dataset>
Dimensions:              (channels: 10, ids: 245, modes: 100)
Coordinates:
  * ids                  (ids) object '8db9e569abe9' ... '18a47a7c3746'
Dimensions without coordinates: channels, modes
Data variables:
    _exceptions          (ids, channels) object '' '' nan nan ... '' nan nan nan
    _runtimes            (ids, channels) float64 37.45 37.45 nan ... nan nan nan
    cutoff               (ids, channels) float64 111.6 111.6 nan ... nan nan nan
    damping_a            (ids, modes, channels) float64 0.04043 0.04043 ... nan
    dec_rate             (ids, channels) float64 44.0 44.0 nan ... nan nan nan
    frequencies_a        (ids, modes, channels) float64 5.95 5.95 ... nan nan
    fs                   (ids, channels) float64 441.3 441.3 nan ... nan nan nan
    margin               (ids, channels) float64 0.7538 0.7424 nan ... nan nan
    modal_contributions  (ids, modes, channels) float64 3.051e-05 ... nan
    modal_damping        (ids, modes, channels) float64 46.41 46.41 ... nan nan
    modal_frequencies    (ids, modes, channels) float64 6.546 6.546 ... nan nan
    mode_shapes          (ids, channels, modes) complex128 (6.004555470391417...
    mode_shapes_a        (ids, channels, modes) complex128 (0.000703590593318...
    model_order          (ids, channels) float64 34.0 34.0 nan ... nan nan nan
    numtaps              (ids, channels) float64 1.188e+03 1.188e+03 ... nan nan
    sim_steps            (ids, channels) float64 2.627e+06 2.627e+06 ... nan nan
    snr_alias            (ids, channels) float64 9.071e+04 7.266e+04 ... nan nan
    snr_db_out           (ids, channels) float64 6.333 6.339 nan ... nan nan nan
    snr_quant            (ids, channels) float64 52.39 52.49 nan ... nan nan nan
Attributes:
    date:         datetime.date(2021, 9, 21)
    dbfile_in:    uq_acqui.nc
    dbfile_out:   uq_acqui_out.nc
    entropy:      989225908
    result_dir:   /usr/scratch4/sima9999/work/modal_uq/uq_acqui
    title:        uq_acqui
    working_dir:  /dev/shm/womo1998/